Let's now try it with the Actor outputs

In [1]:

import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm as tqdm
tf.keras.backend.set_floatx('float32')
from collections import deque
from datetime import datetime
import random
import matplotlib
from environment import Environment
from plots import just_plot
# from plots import *
from nets import *
from buffer import ReplayBuffer
from misc import *
import numpy as np

dolinar_layers=2
amplitude = 0.4
number_phases=2
total_episodes = 10**3
buffer_size=500
batch_size=64
ep_guess=0.01
noise_displacement=0.5
lr_actor=0.01
lr_critic=0.001
tau=0.005



class PolicyEvaluator(Basics):
    def __init__(self, **kwargs):
        amplitude= kwargs.get("amplitude", .4)
        dolinar_layers=kwargs.get("dolinar_layers", 2)
        number_phases=kwargs.get("number_phases", 2)
        super().__init__(amplitude=amplitude, dolinar_layers=dolinar_layers, number_phases=number_phases)
        
        displacement_tree = {}
        #self.at = make_attenuations(self.number_layers)
        for layer in range(self.dolinar_layers+1):
            displacement_tree[str(layer)] = {}

        for k in outcomes_universe(self.dolinar_layers):
            for layer in range(self.dolinar_layers+1):
                displacement_tree[str(layer)][str(k[:layer])] = 0
                
        self.displacement_tree = displacement_tree
        
    def random_tree(self):
        actions = self.displacement_tree.copy()
        for k in outcomes_universe(self.dolinar_layers):
            for layer in range(self.dolinar_layers+1):
                actions[str(layer)][str(k[:layer])] = np.random.random()
        return actions
    
    def success_probability(self, displacements_tree):
        """
        Given a tree of conditional actions (on the outcomes history), computes
        the success probability. Notice the final action is the guess
        for the phase of the state given a given branch.
        """
        p=0
        for ot in outcomes_universe(self.dolinar_layers):
            c=1
            for layer in range(self.dolinar_layers):
                eff_at = np.prod(np.sin(self.at[:layer]))*np.cos(self.at[layer])
                c*=P(displacements_tree[str(self.dolinar_layers)][str(ot)]*self.amplitude,
                     displacements_tree[str(layer)][str(ot[:(layer)])], eff_at, ot[self.dolinar_layers-1-layer] ) #notice i respect that the columns of the outcomes_universe
                #correspond to the layer: the last column is the first layer.
            p += c
        return p/self.number_phases
    
    
    def greedy_strategy(self, actor, critic):
        """Assuming actor, critic and self have the same dolinar_layers.
            self.possible_phases are the possible phases of the coherent states

        """
        rr = np.ones((2**(self.dolinar_layers-1), self.dolinar_layers, 1))*actor.pad_value
        rr[:,1:] = np.reshape(outcomes_universe(self.dolinar_layers-1),(2**(self.dolinar_layers-1), self.dolinar_layers-1,1))
        preds = np.squeeze(actor(rr))
        
        for ot, seqot in zip(outcomes_universe(self.dolinar_layers-1), preds):
            for layer in range(self.dolinar_layers):
                self.displacement_tree[str(layer)][str(ot[:layer])] = seqot[layer]

            history = []
            index_seqot, index_ot= 0, 0
            for index_history in range(2*self.dolinar_layers-1):
                if index_history%2==0:
                    history.append(seqot[index_seqot])
                    index_seqot+=1
                else:
                    history.append(ot[index_ot])
                    index_ot+=1   
            for final_outcome in [0,1]:
                final_history = np.append(history, final_outcome)
                self.displacement_tree[str(self.dolinar_layers)][str(np.append(ot,final_outcome))] = self.possible_phases[critic.give_favourite_guess(final_history)[0]]

        return self.success_probability(self.displacement_tree)

In [2]:

class PolicyEvaluator(Basics):
    def __init__(self, **kwargs):
        amplitude= kwargs.get("amplitude", .4)
        dolinar_layers=kwargs.get("dolinar_layers", 2)
        number_phases=kwargs.get("number_phases", 2)
        super().__init__(amplitude=amplitude, dolinar_layers=dolinar_layers, number_phases=number_phases)
        
        displacement_tree = {}
        #self.at = make_attenuations(self.number_layers)
        for layer in range(self.dolinar_layers+1):
            displacement_tree[str(layer)] = {}

        for k in outcomes_universe(self.dolinar_layers):
            for layer in range(self.dolinar_layers+1):
                displacement_tree[str(layer)][str(k[:layer])] = 0
                
        self.displacement_tree = displacement_tree
        
    def random_tree(self):
        actions = self.displacement_tree.copy()
        for k in outcomes_universe(self.dolinar_layers):
            for layer in range(self.dolinar_layers+1):
                actions[str(layer)][str(k[:layer])] = np.random.random()
        return actions
    
    def success_probability(self, displacements_tree):
        """
        Given a tree of conditional actions (on the outcomes history), computes
        the success probability. Notice the final action is the guess
        for the phase of the state given a given branch.
        """
        p=0
        for ot in outcomes_universe(self.dolinar_layers):
            c=1
            for layer in range(self.dolinar_layers):
                eff_at = np.prod(np.sin(self.at[:layer]))*np.cos(self.at[layer])
                c*=P(displacements_tree[str(self.dolinar_layers)][str(ot)]*self.amplitude,
                     displacements_tree[str(layer)][str(ot[:(layer)])], eff_at, ot[self.dolinar_layers-1-layer] ) #notice i respect that the columns of the outcomes_universe
                #correspond to the layer: the last column is the first layer.
            p += c
        return p/self.number_phases
    
    
    def greedy_strategy(self, actor, critic):
        """Assuming actor, critic and self have the same dolinar_layers.
            self.possible_phases are the possible phases of the coherent states

        """
        rr = np.ones((2**(self.dolinar_layers-1), self.dolinar_layers, 1))*actor.pad_value
        rr[:,1:] = np.reshape(outcomes_universe(self.dolinar_layers-1),(2**(self.dolinar_layers-1), self.dolinar_layers-1,1))
        preds = np.squeeze(actor(rr))
        
        for ot, seqot in zip(outcomes_universe(self.dolinar_layers-1), preds):
            for layer in range(self.dolinar_layers):
                self.displacement_tree[str(layer)][str(ot[:layer])] = seqot[layer]

            history = []
            index_seqot, index_ot= 0, 0
            for index_history in range(2*self.dolinar_layers-1):
                if index_history%2==0:
                    history.append(seqot[index_seqot])
                    index_seqot+=1
                else:
                    history.append(ot[index_ot])
                    index_ot+=1   
            for final_outcome in [0,1]:
                final_history = np.append(history, final_outcome)
                self.displacement_tree[str(self.dolinar_layers)][str(np.append(ot,final_outcome))] = self.possible_phases[critic.give_favourite_guess(final_history)[0]]

        return self.success_probability(self.displacement_tree)

In [3]:
outcomes_universe(1)

array([[0],
       [1]])

In [4]:
dolinar_layers=2

policy_evaluator = PolicyEvaluator(amplitude = amplitude, dolinar_layers=dolinar_layers, number_phases = number_phases)
env = Environment(amplitude=0.4, dolinar_layers = dolinar_layers)
buffer = ReplayBuffer(buffer_size=buffer_size)

critic = Critic(nature="primary",valreg=0.01, dolinar_layers = dolinar_layers, number_phases=number_phases)
critic_target = Critic(nature="target", dolinar_layers = dolinar_layers, number_phases=number_phases)
actor = Actor(nature="primary", dolinar_layers = dolinar_layers)
actor_target = Actor(nature="target", dolinar_layers = dolinar_layers)


rt = []
pt = []
new_loss=0.

In [5]:
for episode in tqdm(range(10)):

    env.pick_phase()
    experiences=[] #where the current history of the current episode is stored
    context_outcome_actor = np.reshape(np.array([actor.pad_value]),(1,1,1)).astype(np.float32)
    for layer in range(actor.dolinar_layers):
        beta_would_do = np.squeeze(actor(context_outcome_actor))
        beta =  beta_would_do + np.random.uniform(-noise_displacement, noise_displacement)
        outcome = env.give_outcome(beta,layer)
        experiences.append(beta)
        experiences.append(outcome)

        context_outcome_actor = np.reshape(np.array([outcome]),(1,1,1)).astype(np.float32)

    ### ep-gredy guessing of the phase###
    if np.random.random()< ep_guess:
        val = np.random.choice(range(number_phases),1)[0]
        guess_index, guess_input_network = val, val/critic.number_phases
    else:
        guess_index, guess_input_network = critic.give_favourite_guess(experiences) #experiences is the branch of the current tree of actions + outcomes.
    experiences.append(guess_input_network)

    reward = env.give_reward(guess_input_network)
    experiences.append(reward)
    buffer.add(tuple(experiences))
    
    actor.lstm.stateful=False
    pt.append(policy_evaluator.greedy_strategy(actor = actor, critic = critic))
    actor.lstm.stateful=True

  0%|          | 0/10 [00:00<?, ?it/s]


If you intended to run this layer in float64, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float32 by default, call `tf.keras.backend.set_floatx('float32')`. To change just this layer, pass dtype='float32' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



100%|██████████| 10/10 [00:01<00:00,  5.67it/s]


Let's try to put some tree which is more or less meaningful...

In [8]:
policy_evaluator.displacement_tree

{'0': {'[]': 0.033168748},
 '1': {'[0]': 0.033168748, '[1]': 0.033349838},
 '2': {'[0 0]': (-1+0j), '[0 1]': (1+0j), '[1 0]': (-1+0j), '[1 1]': (1+0j)}}

In [26]:
policy_evaluator.displacement_tree = {"0":{"[]":-.7}, "1":{"[0]": -.6, "[1]": .6}, "2":{"[0 0]": 1, "[0 1]":1, "[1 0]":1, "[1 1]":1}}

In [27]:
policy_evaluator.displacement_tree

{'0': {'[]': -0.7},
 '1': {'[0]': -0.6, '[1]': 0.6},
 '2': {'[0 0]': 1, '[0 1]': 1, '[1 0]': 1, '[1 1]': 1}}

In [28]:
policy_evaluator.success_probability(policy_evaluator.displacement_tree)

0.7228144406306147

In [ ]:
policy_evaluator.dolinar_layers

In [ ]:
def greedy_strategy(self, actor, critic):
    """Assuming actor, critic and self have the same dolinar_layers.
        self.possible_phases are the possible phases of the coherent states
    
    """
    rr = np.ones((2**(self.dolinar_layers-1), self.dolinar_layers, 1))*actor.pad_value
    rr[:,1:] = np.reshape(outcomes_universe(actor.dolinar_layers-1),(2**(self.dolinar_layers-1), self.dolinar_layers-1,1))
    preds = np.squeeze(actor(rr))

    for ot, seqot in zip(outcomes_universe(actor.dolinar_layers-1), preds):
        for layer in range(actor.dolinar_layers):
            self.displacement_tree[str(layer)][str(ot[:layer])] = seqot[layer]

        history = []
        index_seqot, index_ot= 0, 0
        for index_history in range(2*actor.dolinar_layers-1):
            if index_history%2==0:
                history.append(seqot[index_seqot])
                index_seqot+=1
            else:
                history.append(ot[index_ot])
                index_ot+=1   
        for final_outcome in [0,1]:
            final_history = np.append(history, final_outcome)
            self.displacement_tree[str(actor.dolinar_layers)][str(np.append(ot,final_outcome))] = self.possible_phases[critic.give_favourite_guess(final_history)[0]]
            
    return self.success_probability(displacement_tree)

In [ ]:
actions = Actions(number_layers=3, number_phases=2)

In [ ]:
actions.success_probability(displacement_tree)

In [ ]:
critic.dolinar_layers

In [ ]:
possible_phases = env.possible_phases